# Pourcentage de satisfaction des pages `contributions` génériques et personnalisées

Dans cette exploration, le but est de récupérer le pourcentage de personne qui ont répondue `Oui` ou `Non` à la question : `Avez-vous trouvé la réponse à votre question ?`

Nous allons appliquer cette analyse à chacune des pages `contributions` génériques et personnalisées.

## 1. Chargement des librairies

In [ ]:
import pandas as pd
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 1000)

## 2. Récupération des queries sur elasticsearch

In [ ]:
es_connector = ElasticsearchConnector(env='monolog')

QUERY_LOG_CONTRIB = {
    "query": {
        "bool": { 
          "must": [
            {
              "prefix": {
                "url": "https://code.travail.gouv.fr/contribution" 
              }
            },
            {
              "range": {
               "logfile": {
                    "gte": "2023-09-01",
                    "lte": "2023-10-01"
               }
              }
            }
          ]
        }
      }
}

In [ ]:
logs = es_connector.execute_query(QUERY_LOG_CONTRIB, "logs-new")

## 3. Vue d'ensemble

### Analyse des contributions génériques et conventionnelles

In [ ]:
# Récupération de l'ensemble des feedback
logs_feedback = logs[logs["type"] == "feedback"] 

In [ ]:
# Nombre total de feedback
logs_feedback["feedbackType"].count()

In [ ]:
# Nombre total de feedback négatif
logs_feedback[logs_feedback["feedbackType"] == "negative"]["feedbackType"].count()

In [ ]:
# Nombre total de feedback positif
logs_feedback[logs_feedback["feedbackType"] == "positive"]["feedbackType"].count()

In [ ]:
# Pourcentage d'avis positive
logs_feedback[logs_feedback["feedbackType"] == "positive"]["feedbackType"].count() / logs_feedback["feedbackType"].count() * 100

In [ ]:
# Pourcentage d'avis négatif
logs_feedback[logs_feedback["feedbackType"] == "negative"]["feedbackType"].count() / logs_feedback["feedbackType"].count() * 100

### Pour les contenus génériques

In [ ]:
# Récupération des logs des urls de contribution génériques
logs_feedback_generic = logs_feedback[~logs_feedback["url"].str.contains("contribution/[0-9]{1,4}-")]

In [ ]:
# Nombre total de feedback
logs_feedback_generic["feedbackType"].count()

In [ ]:
# Nombre total de feedback négatif
logs_feedback_generic[logs_feedback_generic["feedbackType"] == "negative"]["feedbackType"].count()

In [ ]:
# Nombre total de feedback positif
logs_feedback_generic[logs_feedback_generic["feedbackType"] == "positive"]["feedbackType"].count()

In [ ]:
# Pourcentage d'avis positive
logs_feedback_generic[logs_feedback_generic["feedbackType"] == "positive"]["feedbackType"].count() / logs_feedback_generic["feedbackType"].count() * 100

In [ ]:
# Pourcentage d'avis négatif
logs_feedback_generic[logs_feedback_generic["feedbackType"] == "negative"]["feedbackType"].count() / logs_feedback_generic["feedbackType"].count() * 100

In [ ]:
# Nombre total de feedback
logs_feedback_generic["feedbackType"].count()

In [ ]:
# Nombre total de feedback négatif
logs_feedback_generic[logs_feedback_generic["feedbackType"] == "negative"]["feedbackType"].count()

In [ ]:
# Nombre total de feedback positif
logs_feedback_generic[logs_feedback_generic["feedbackType"] == "positive"]["feedbackType"].count()

In [ ]:
# Pourcentage d'avis positive
logs_feedback_generic[logs_feedback_generic["feedbackType"] == "positive"]["feedbackType"].count() / logs_feedback_generic["feedbackType"].count() * 100

In [ ]:
# Pourcentage d'avis négatif
logs_feedback_generic[logs_feedback_generic["feedbackType"] == "negative"]["feedbackType"].count() / logs_feedback_generic["feedbackType"].count() * 100

### Pour les contenus liés aux conventions collectives

In [ ]:
# Récupération des logs des urls de contribution liées à une convention collective
logs_feedback_ccs = logs_feedback[logs_feedback["url"].str.contains("contribution/[0-9]{1,4}-")]

In [ ]:
# Nombre total de feedback
logs_feedback_ccs["feedbackType"].count()

In [ ]:
# Nombre total de feedback négatif
logs_feedback_ccs[logs_feedback_ccs["feedbackType"] == "negative"]["feedbackType"].count()

In [ ]:
# Nombre total de feedback positif
logs_feedback_ccs[logs_feedback_ccs["feedbackType"] == "positive"]["feedbackType"].count()

In [ ]:
# Pourcentage d'avis positive
logs_feedback_ccs[logs_feedback_ccs["feedbackType"] == "positive"]["feedbackType"].count() / logs_feedback_ccs["feedbackType"].count() * 100

In [ ]:
# Pourcentage d'avis négatif
logs_feedback_ccs[logs_feedback_ccs["feedbackType"] == "negative"]["feedbackType"].count() / logs_feedback_ccs["feedbackType"].count() * 100

## Vue pour chaque page

### Contributions génériques

In [ ]:
logs_feedback_generic_grouped = logs_feedback_generic.groupby(["url", "feedbackType"]).size().reset_index(name='counts').sort_values(by=['url'])

logs_feedback_generic_grouped["ratio"] = logs_feedback_generic_grouped["counts"] / logs_feedback_generic_grouped.groupby("url")["counts"].transform('sum') * 100

logs_feedback_generic_grouped

### Pour les conventions collectives

In [ ]:
logs_feedback_ccs_grouped = logs_feedback_ccs.groupby(["url", "feedbackType"]).size().reset_index(name='counts').sort_values(by=['url'])

# get the ratio of each group per url
logs_feedback_ccs_grouped["ratio"] = logs_feedback_ccs_grouped["counts"] / logs_feedback_ccs_grouped.groupby("url")["counts"].transform('sum') * 100

logs_feedback_ccs_grouped

## Vue groupée par convention collective et contenu générique

### Génériques

In [ ]:
logs_feedback_generic_grouped_summarized = logs_feedback_generic.groupby(["feedbackType"]).size().reset_index(name='counts')

logs_feedback_generic_grouped_summarized["ratio"] = logs_feedback_generic_grouped_summarized["counts"] / logs_feedback_generic_grouped_summarized["counts"].sum() * 100

logs_feedback_generic_grouped_summarized

### Conventions collectives

In [ ]:
logs_feedback_ccs_grouped_summarized = logs_feedback_ccs.groupby(["feedbackType"]).size().reset_index(name='counts')

logs_feedback_ccs_grouped_summarized["ratio"] = logs_feedback_ccs_grouped_summarized["counts"] / logs_feedback_ccs_grouped_summarized["counts"].sum() * 100

logs_feedback_ccs_grouped_summarized